# TAO SSD

TechGo is a project focused on developing an automatic store with object detection models playing a crucial role in identifying user-selected products. A carefully curated dataset consisting of 1397 training samples, 400 validation samples, and 200 testing samples was constructed. The dataset includes six specific labels, namely Coca Cola, Pringles, Doritos, M&M, hands, and person, representing common objects and interactions within a store environment.

NOTE: This notebooks is an adaptation of the official one modificated for TechGo project. Official: (https://github.com/NVIDIA-AI-IOT/nvidia-tao/tree/main/tensorflow/ssd)

## Switch to CPU Instance (Advisable only for Non Colab-Pro instance)

1. Switch to CPU Instance for until Step 2 for non GPU dependent tasks
2. This increases your time available for the GPU dependent tasks on a Colab instance
2. Change Runtime type to CPU by Runtime(Top Left tab)->Change Runtime Type->None(Hardware Accelerator)
3.   Then click on Connect (Top Right)



## Mounting Google drive
Mount your Google drive storage to this Colab instance

In [1]:
try:
    import google.colab
    %env GOOGLE_COLAB=1
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
except:
    %env GOOGLE_COLAB=0
    print("Warning: Not a Colab Environment")

env: GOOGLE_COLAB=1
Mounted at /content/drive


# Object Detection using TAO SSD

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task.

Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://developer.nvidia.com/sites/default/files/akamai/TAO/tlt-tao-toolkit-bring-your-own-model-diagram.png" width="1080">

## Learning Objectives
In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Take a pretrained resnet18 model and train a ResNet-18 SSD model on the TechGo dataset
* Prune the trained SSD model
* Retrain the pruned model to recover lost accuracy
* Export the pruned model
* Run Inference on the trained model
* Export the pruned, quantized and retrained model to a .etlt file for deployment to DeepStream


## 0. Set up env variables and set FIXME parameters <a class="anchor" id="head-0"></a>

In [ ]:
# Setting up env variables for cleaner command line commands.
import os

%env TAO_DOCKER_DISABLE=1

%env KEY=nvidia_tlt
#FIXME1
%env NUM_GPUS=1
#FIXME2
%env GPU_INDEX=0

#FIXME3 Where you want to save the official repository in your google drive
%env COLAB_NOTEBOOKS_PATH=/content/drive/MyDrive/perceptionAI/TechGo/nvidia-tao
if os.environ["GOOGLE_COLAB"] == "1":
    if not os.path.exists(os.path.join(os.environ["COLAB_NOTEBOOKS_PATH"])):
      !git clone https://github.com/NVIDIA-AI-IOT/nvidia-tao.git $COLAB_NOTEBOOKS_PATH
else:
    if not os.path.exists(os.environ["COLAB_NOTEBOOKS_PATH"]):
        raise Exception("Error, enter the path of the colab notebooks repo correctly")

#FIXME4 set this path to a folder location where pretrained models, checkpoints and log files during different model actions will be saved
%env EXPERIMENT_DIR=/content/drive/MyDrive/perceptionAI/TechGo/TAO/results/ssd
#FIXME5
delete_existing_experiments = False
#FIXME6 set this path to a folder location where you want to dataset to be present
%env DATA_DIR=/content/drive/MyDrive/perceptionAI/TechGo/data/TechKITTI/
#FIXME7
delete_existing_data = False

if delete_existing_experiments:
    !sudo rm -rf $EXPERIMENT_DIR
if delete_existing_data:
    !sudo rm -rf $DATA_DIR

SPECS_DIR=f"{os.environ['COLAB_NOTEBOOKS_PATH']}/tensorflow/ssd/specs"
%env SPECS_DIR={SPECS_DIR}
# Showing list of specification files.
!ls -rlt $SPECS_DIR

!sudo mkdir -p $DATA_DIR && sudo chmod -R 777 $DATA_DIR
!sudo mkdir -p $EXPERIMENT_DIR && sudo chmod -R 777 $EXPERIMENT_DIR

## 1. Prepare dataset and pre-trained model <a class="anchor" id="head-1"></a>

### 1.1 Transform to KITTI format <a class="anchor" id="head-1-1"></a>
By default TAO toolkit works with KITTI or COCO dataset format. In this notebook we will work in transforming yolov7 format to KITTI

In [ ]:
from pathlib import Path
import shutil
from sklearn.model_selection import train_test_split

# Dataset anotations for KITTI
# https://docs.nvidia.com/tao/tao-toolkit/text/data_annotation_format.html#id4


class YOLOtoKITTI:
    """
    The purpose of this class is to Transform YOLOV7 format to KITTI format
    """
    def __init__(self, source: str, destination: str):
        """
        Initializes a new instance of the YOLOtoKITTI class.

        Args:
            source (str): The source directory path containing YOLOV7 format data.
            destination (str): The destination directory path for storing the transformed KITTI format data.
        """
        self.source = Path(source)
        self.destination = Path(destination)

        for split in ['train', 'test', 'valid']:
            destination_folder = self.destination / split / 'labels'
            destination_folder.mkdir(parents=True, exist_ok=True)


    def __call__(self, width, height, class_names, dataset_name):
        """
        Transforms YOLOV7 format data to KITTI format.

        Args:
            width (int): The width of the images in pixels.
            height (int): The height of the images in pixels.
            class_names (dict): a dict of classes
            dataset_name (str): The name that I want to use for my transformed dataset
        """

        for folder in self.source.iterdir():
            folder = folder / "labels"
            for fnames in folder.glob('*'):
                data=[]
                with open(str(fnames), 'r') as fh:

                    data=fh.readlines()

                for cc, lines in enumerate(data):
                    lines=lines.replace('\n', '').split(" ")

                    n_line = [float(0)] * 15
                    n_line[0] = class_names[int(lines[0])]
                    # n_line[1] = float(0)
                    n_line[2] = int(0)

                    x, y, w, h = float(lines[1]), float(lines[2]), float(lines[3]), float(lines[4])
                    x, y, w, h = x*width, y*height, w*width, h*height
                    x_min, y_min = int(x - w/2), int(y - h/2)
                    x_max, y_max = int(x + w/2), int(y + h/2)

                    n_line[4], n_line[5], n_line[6], n_line[7] = float(x_min), float(y_min), float(x_max), float(y_max)

                    n_line = [float(1) if i > 7 else val for i, val in enumerate(n_line)]

                    str1 = ' '.join(str(n_line)).replace(' ', '').replace(',', ' ').replace('[', '').replace("]","").replace("'","").replace('"',"")
                    data[cc]= str1 +"\n"

                strf = " ".join(str(x) for x in data)

                strf=strf.replace("\n ","\n")

                file = open(str(fnames).replace('Yolov7Format', dataset_name), "w")
                file.write(strf)
                file.close()
        print("Labels created!")

        # Get images
        for source_path in self.source.iterdir():
            source_path = source_path / "images"
            destination_path = str(source_path).replace('Yolov7Format',dataset_name)

            if shutil.os.path.exists(destination_path):
                shutil.rmtree(destination_path)

            shutil.copytree(source_path, destination_path)
        print("Images created!")

In [ ]:
pipe = YOLOtoKITTI(
    source='/content/drive/MyDrive/perceptionAI/TechGo/data/Yolov7Format',
    destination='/content/drive/MyDrive/perceptionAI/TechGo/data/TechKITTI'
)

In [ ]:
class_names = {
    0: 'coca-cola',
    1: 'm&m',
    2: 'pringles',
    3: 'doritos',
    4: 'person',
    5: 'hand'
}

pipe(
    width=1920,
    height=1080,
    class_names=class_names,
    dataset_name='TechKITTI'
)


### 1.2 Download pre-trained model <a class="anchor" id="head-1-1"></a>

We will use NGC CLI to get the pre-trained models. For more details, go to [ngc.nvidia.com](ngc.nvidia.com) and click the SETUP on the navigation bar.

In [ ]:
# Installing NGC CLI on the local machine.
## Download and install
%env LOCAL_PROJECT_DIR=/ngc_content/
%env CLI=ngccli_cat_linux.zip
!sudo mkdir -p $LOCAL_PROJECT_DIR/ngccli && sudo chmod -R 777 $LOCAL_PROJECT_DIR

# Remove any previously existing CLI installations
!sudo rm -rf $LOCAL_PROJECT_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $LOCAL_PROJECT_DIR/ngccli
!unzip -u -q "$LOCAL_PROJECT_DIR/ngccli/$CLI" -d $LOCAL_PROJECT_DIR/ngccli/
!rm $LOCAL_PROJECT_DIR/ngccli/*.zip
os.environ["PATH"]="{}/ngccli/ngc-cli:{}".format(os.getenv("LOCAL_PROJECT_DIR", ""), os.getenv("PATH", ""))
!cp /usr/lib/x86_64-linux-gnu/libstdc++.so.6 $LOCAL_PROJECT_DIR/ngccli/ngc-cli/libstdc++.so.6

In [ ]:
!ngc registry model list nvidia/tao/pretrained_object_detection:*

In [ ]:
!mkdir -p $EXPERIMENT_DIR/pretrained_resnet18/

In [ ]:
# Pull pretrained model from NGC
!ngc registry model download-version nvidia/tao/pretrained_object_detection:resnet10 --dest $EXPERIMENT_DIR/pretrained_resnet10

In [ ]:
print("Check that model is downloaded into dir.")
!ls -l $EXPERIMENT_DIR/pretrained_resnet10/pretrained_object_detection_vresnet10

## 2. Setup GPU environment <a class="anchor" id="head-2"></a>


### 2.1 Connect to GPU Instance <a class="anchor" id="head-2-1"></a>

1. Move any data saved to the Colab Instance storage to Google Drive
2. Change Runtime type to GPU by Runtime(Top Left tab)->Change Runtime Type->GPU(Hardware Accelerator)
3.   Then click on Connect (Top Right)



### 2.2 Mounting Google drive <a class="anchor" id="head-2-2"></a>
Mount your Google drive storage to this Colab instance

In [ ]:
try:
    import google.colab
    %env GOOGLE_COLAB=1
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
except:
    %env GOOGLE_COLAB=0
    print("Warning: Not a Colab Environment")

### 2.3 Setup Python environment <a class="anchor" id="head-2-3"></a>
Setup the environment necessary to run the TAO Networks by running the bash script

In [ ]:
import os
if os.environ["GOOGLE_COLAB"] == "1":
    os.environ["bash_script"] = "setup_env.sh"
else:
    os.environ["bash_script"] = "setup_env_desktop.sh"

!sed -i "s|PATH_TO_COLAB_NOTEBOOKS|$COLAB_NOTEBOOKS_PATH|g" $COLAB_NOTEBOOKS_PATH/tensorflow/$bash_script

!sh $COLAB_NOTEBOOKS_PATH/tensorflow/$bash_script

In [ ]:
if os.environ.get("PYTHONPATH","") == "":
    os.environ["PYTHONPATH"] = ""
os.environ["PYTHONPATH"]+=":/opt/nvidia/"
if os.environ["GOOGLE_COLAB"] == "1":
    os.environ["PYTHONPATH"]+=":/usr/local/lib/python3.6/dist-packages/third_party/nvml"
else:
    os.environ["PYTHONPATH"]+=":/home_duplicate/rarunachalam/miniconda3/envs/tf_py_36/lib/python3.6/site-packages/third_party/nvml" # FIX MINICONDA PATH

In [ ]:
# Reset NVIDIA-DALI version for SSD
if os.environ["GOOGLE_COLAB"] == "1":
    !python3.6 -m pip uninstall nvidia-dali-nvtf-plugin -y
    !python3.6 -m pip install --extra-index-url https://developer.download.nvidia.com/compute/redist --upgrade nvidia-dali-cuda110==0.31.0
    !python3.6 -m pip install --extra-index-url https://developer.download.nvidia.com/compute/redist --upgrade nvidia-dali-tf-plugin-cuda110==0.31.0

### 2.4 Reset env variables (Use the same paths which was set in Step 0) <a class="anchor" id="head-2-4"></a>

In [ ]:
# Setting up env variables for cleaner command line commands.
import os

%env TAO_DOCKER_DISABLE=1

%env KEY=nvidia_tlt
%env NUM_GPUS=1
%env GPU_INDEX=0

# Change the paths according to your directory structure, these are just examples
%env COLAB_NOTEBOOKS_PATH=/content/drive/MyDrive/perceptionAI/TechGo/nvidia-tao
if not os.path.exists(os.environ["COLAB_NOTEBOOKS_PATH"]):
    raise Exception("Error, enter the path of the colab notebooks repo correctly")
%env EXPERIMENT_DIR=/content/drive/MyDrive/perceptionAI/TechGo/TAO/results/ssd
%env DATA_DIR=/content/drive/MyDrive/perceptionAI/TechGo/data/TechKITTI/

SPECS_DIR=f"{os.environ['COLAB_NOTEBOOKS_PATH']}/tensorflow/ssd/specs"
%env SPECS_DIR={SPECS_DIR}
# Showing list of specification files.
!ls -rlt $SPECS_DIR

## 3. Generate tfrecords <a class="anchor" id="head-3"></a>

The default SSD data format requires generation of TFRecords.
Now we need to modified the proportionated txt files in order to set the tfrecords transformation:
- ssd_tfrecords_kitti_train.txt
- ssd_tfrecords_kitti_val.txt
- ssd_tfrecords_kitti_test.txt (Additional)

Complete them as follows changing the value of "train", "valid", "test":

```
kitti_config {
  root_directory_path: "/content/drive/MyDrive/perceptionAI/TechGo/data/TechKITTI///train"
  image_dir_name: "images"
  label_dir_name: "labels"
  image_extension: ".jpg"
  partition_mode: "random"
  num_partitions: 2
  val_split: 0
  num_shards: 10
}
image_directory_path: "/content/drive/MyDrive/perceptionAI/TechGo/data/TechKITTI///train"
target_class_mapping {
      key: "m&m"
      value: "m&m"
  }
target_class_mapping {
    key: "doritos"
    value: "doritos"
}
target_class_mapping {
    key: "coca-cola"
    value: "coca-cola"
}
target_class_mapping {
    key: "pringles"
    value: "pringles"
}
target_class_mapping {
    key: "person"
    value: "person"
}
target_class_mapping {
    key: "hand"
    value: "hand"
}
```

In [ ]:
# Creating a new directory for the output tfrecords dump.
print("Converting the training set to TFRecords.")
!mkdir -p $DATA_DIR/tfrecords_ssd/kitti_train && sudo rm -rf $DATA_DIR/tfrecords_ssd/kitti_train/*
!tao ssd dataset_convert -d $SPECS_DIR/ssd_tfrecords_kitti_train.txt \
                         -o $DATA_DIR/tfrecords_ssd/kitti_train/ktrain

In [ ]:
# Creating a new directory for the output tfrecords dump.
print("Converting the validation set to TFRecords.")
!mkdir -p $DATA_DIR/tfrecords_ssd/kitti_val && sudo rm -rf $DATA_DIR/tfrecords_ssd/kitti_val/*
!tao ssd dataset_convert \
         -d $SPECS_DIR/ssd_tfrecords_kitti_val.txt \
         -o $DATA_DIR/tfrecords_ssd/kitti_val/kval

In [ ]:
# Creating a new directory for the output tfrecords dump.
print("Converting the testing set to TFRecords.")
!mkdir -p $DATA_DIR/tfrecords_ssd/kitti_test && sudo rm -rf $DATA_DIR/tfrecords_ssd/kitti_test/*
!tao ssd dataset_convert \
         -d $SPECS_DIR/ssd_tfrecords_kitti_test.txt \
         -o $DATA_DIR/tfrecords_ssd/kitti_test/ktest

In [ ]:
!ls -rlt $DATA_DIR/tfrecords/

## 4. Provide training specification <a class="anchor" id="head-4"></a>
* Dataset for the train datasets
    * In order to use the newly generated dataset, update the dataset_config parameter in the spec file at `$SPECS_DIR/ssd_train_resnet18_kitti.txt`
* Augmentation parameters for on the fly data augmentation
* Other training (hyper-)parameters such as batch size, number of epochs, learning rate etc.

Important considerations
- In this case, the pretrained weights are set in the "tao ssd train" command
- Remember to set the training size in the aumentation_config section in output_width and output_height. All the training images are going to be resized to that shape.
- Remember to add the dataclasses inside dataset_config section



```
random_seed: 42
ssd_config {
  aspect_ratios_global: "[1.0, 2.0, 0.5, 3.0, 1.0/3.0]"
  scales: "[0.05, 0.1, 0.25, 0.4, 0.55, 0.7, 0.85]"
  two_boxes_for_ar1: true
  clip_boxes: false
  variances: "[0.1, 0.1, 0.2, 0.2]"
  arch: "resnet"
  nlayers: 18
  freeze_bn: false
  freeze_blocks: 0
}
training_config {
  batch_size_per_gpu: 16
  num_epochs: 1
  enable_qat: false
  learning_rate {
  soft_start_annealing_schedule {
    min_learning_rate: 5e-5
    max_learning_rate: 2e-2
    soft_start: 0.15
    annealing: 0.8
    }
  }
  regularizer {
    type: L1
    weight: 3e-5
  }
}
eval_config {
  validation_period_during_training: 10
  average_precision_mode: SAMPLE
  batch_size: 16
  matching_iou_threshold: 0.5
}
nms_config {
  confidence_threshold: 0.01
  clustering_iou_threshold: 0.6
  top_k: 200
}
augmentation_config {
    output_width: 300
    output_height: 300
    output_channel: 3
}
dataset_config {
  data_sources: {
    tfrecords_path: "/content/drive/MyDrive/perceptionAI/TechGo/data/TechKITTI/tfrecords_ssd/kitti_train/ktrain*"
    # label_directory_path: "/content/drive/MyDrive/perceptionAI/TechGo/data/TechKITTI///train/labels"
    # image_directory_path: "/content/drive/MyDrive/perceptionAI/TechGo/data/TechKITTI///train/images"
  }
  include_difficult_in_training: true
  target_class_mapping {
      key: "m&m"
      value: "m&m"
  }
  target_class_mapping {
      key: "doritos"
      value: "doritos"
  }
  target_class_mapping {
      key: "coca-cola"
      value: "coca-cola"
  }
  target_class_mapping {
      key: "pringles"
      value: "pringles"
  }
  target_class_mapping {
      key: "person"
      value: "person"
  }
  target_class_mapping {
      key: "hand"
      value: "hand"
  }
  validation_data_sources: {
      # tfrecords_path: "/content/drive/MyDrive/perceptionAI/TechGo/data/TechKITTI/tfrecords_ssd/kitti_val/kval*"
      label_directory_path: "/content/drive/MyDrive/perceptionAI/TechGo/data/TechKITTI///valid/labels"
      image_directory_path: "/content/drive/MyDrive/perceptionAI/TechGo/data/TechKITTI///valid/images"
  }
}
```

## 5. Run TAO training <a class="anchor" id="head-5"></a>
* Provide the sample spec file and the output directory location for models
* WARNING: training will take several hours or one day to complete

In [ ]:
!mkdir -p $EXPERIMENT_DIR/experiment_dir_unpruned/1280

In [ ]:
print("To run with multigpu, please change --gpus based on the number of available GPUs in your machine.")
!tao ssd train --gpus 1 --gpu_index=$GPU_INDEX \
               -e $SPECS_DIR/300/ssd_train_resnet10_kitti.txt \
               -r $EXPERIMENT_DIR/experiment_dir_unpruned/300 \
               -k $KEY \
               -m $EXPERIMENT_DIR/pretrained_resnet10/pretrained_object_detection_vresnet10/resnet_10.hdf5

In [ ]:
print('Model for each epoch:')
print('---------------------')
!ls -ltrh $EXPERIMENT_DIR/experiment_dir_unpruned/300/weights

In [ ]:
# Now check the evaluation stats in the csv file and pick the model with highest eval accuracy.
!cat $EXPERIMENT_DIR/experiment_dir_unpruned/300/ssd_training_log_resnet10.csv
%env EPOCH=010

## 6. Evaluate trained models <a class="anchor" id="head-6"></a>

As default, the "evaluate command" runs the inference in the validation data source. As a result, it is recommended to create a copy of the training txt file and change the data validation source to our test set and rename it, for example: "test_set_300.txt"

In [ ]:
!tao ssd evaluate --gpu_index=$GPU_INDEX \
                  -e $SPECS_DIR/300/test_set_300.txt \
                  -m $EXPERIMENT_DIR/experiment_dir_unpruned/300/weights/ssd_resnet10_epoch_$EPOCH.tlt \
                  -k $KEY

## 7. Prune trained models <a class="anchor" id="head-7"></a>
* Specify pre-trained model
* Equalization criterion (`Only for resnets as they have element wise operations or MobileNets.`)
* Threshold for pruning.
* A key to save and load the model
* Output directory to store the model

Usually, you just need to adjust `-pth` (threshold) for accuracy and model size trade off. Higher `pth` gives you smaller model (and thus higher inference speed) but worse accuracy. The threshold value depends on the dataset and the model. `0.5` in the block below is just a start point. If the retrain accuracy is good, you can increase this value to get smaller models. Otherwise, lower this value to get better accuracy.

In [ ]:
!mkdir -p $EXPERIMENT_DIR/experiment_dir_pruned/300

In [ ]:
!tao ssd prune --gpu_index=$GPU_INDEX \
               -m $EXPERIMENT_DIR/experiment_dir_unpruned/300/weights/ssd_resnet10_epoch_$EPOCH.tlt \
               -o $EXPERIMENT_DIR/experiment_dir_pruned/300/ssd_resnet10_pruned.tlt \
               -eq intersection \
               -pth 0.1 \
               -k $KEY

In [ ]:
!ls -rlt $EXPERIMENT_DIR/experiment_dir_pruned/300/

## 8. Retrain pruned models <a class="anchor" id="head-8"></a>
Model needs to be re-trained to bring back accuracy after pruning
Specify re-training specification in the txt retrain file "ssd_retrain_resnet18_kitti_300.txt". It is almost the same as the train file with some changes, for example:
- Learning rate
- Regularized

```
  learning_rate {
    soft_start_annealing_schedule {
      min_learning_rate: 5e-5
      max_learning_rate: 2e-2
      soft_start: 0.15
      annealing: 0.8
      }
  }
  regularizer {
    type: NO_REG
    weight: 3e-9
  }
```

In [ ]:
!mkdir -p $EXPERIMENT_DIR/experiment_dir_retrain/300

In [ ]:
# Retraining using the pruned model as pretrained weights
!tao ssd train --gpus 1 --gpu_index=$GPU_INDEX \
               -e $SPECS_DIR/300/ssd_retrain_resnet10_kitti.txt \
               -r $EXPERIMENT_DIR/experiment_dir_retrain/300/ \
               -m $EXPERIMENT_DIR/experiment_dir_pruned/300/ssd_resnet10_pruned.tlt \
               -k $KEY

In [ ]:
# Listing the newly retrained model.
!ls -rlt $EXPERIMENT_DIR/experiment_dir_retrain/300/weights

In [ ]:
# Now check the evaluation stats in the csv file and pick the model with highest eval accuracy.
!cat $EXPERIMENT_DIR/experiment_dir_retrain/300/ssd_training_log_resnet10.csv
%env EPOCH=010

## 9. Evaluate retrained model <a class="anchor" id="head-9"></a>

In [ ]:
!tao ssd evaluate --gpu_index=$GPU_INDEX \
                  -e $SPECS_DIR/300/test_set_300.txt \
                  -m $EXPERIMENT_DIR/experiment_dir_retrain/300/weights/ssd_resnet10_epoch_$EPOCH.tlt \
                  -k $KEY

## 10. Visualize inferences <a class="anchor" id="head-10"></a>
In this section, we run the `infer` tool to generate inferences on the trained models and visualize the results. It is important to mention that the "Non Maximum Suppression" part is not perform with the model, as a result, the inference will show many bounding boxes for each detected object

In [ ]:
!mkdir -p $EXPERIMENT_DIR/ssd_infer_images
!mkdir -p $EXPERIMENT_DIR/ssd_infer_labels

In [ ]:
# Running inference for detection on n images
!tao ssd inference --gpu_index=$GPU_INDEX -i $DATA_DIR/test/images \
                   -o $EXPERIMENT_DIR/ssd_infer_images \
                   -e $SPECS_DIR/300/ssd_retrain_resnet10_kitti.txt \
                   -m $EXPERIMENT_DIR/experiment_dir_retrain/300/weights/ssd_resnet10_epoch_$EPOCH.tlt \
                   -l $EXPERIMENT_DIR/ssd_infer_labels \
                   -k $KEY

The `tao` inference tool produces two outputs.
1. Overlain images in `$EXPERIMENT_DIR/ssd_infer_images`
2. Frame by frame bbox labels in kitti format located in `$EXPERIMENT_DIR/ssd_infer_labels`

In [ ]:
# Simple grid visualizer
import matplotlib.pyplot as plt
import os
from math import ceil
valid_image_ext = ['.jpg', '.png', '.jpeg', '.ppm']

def visualize_images(image_dir, num_cols=4, num_images=10):
    output_path = os.path.join(os.environ['EXPERIMENT_DIR'], image_dir)
    num_rows = int(ceil(float(num_images) / float(num_cols)))
    f, axarr = plt.subplots(num_rows, num_cols, figsize=[80,30])
    f.tight_layout()
    a = [os.path.join(output_path, image) for image in os.listdir(output_path)
         if os.path.splitext(image)[1].lower() in valid_image_ext]
    for idx, img_path in enumerate(a[:num_images]):
        col_id = idx % num_cols
        row_id = idx // num_cols
        img = plt.imread(img_path)
        axarr[row_id, col_id].imshow(img)

In [ ]:
# Visualizing the sample images.
OUTPUT_PATH = 'ssd_infer_images' # relative path from $EXPERIMENT_DIR.
COLS = 3 # number of columns in the visualizer grid.
IMAGES = 9 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

# Deployment
In this part we will create the ".etlt" file that is going to be our input to deepstream where the ".engine" file will be created.

In [ ]:
%env KEY=nvidia_tlt
#FIXME1
%env NUM_GPUS=1
#FIXME2
%env GPU_INDEX=0
#FIXME3
%env COLAB_NOTEBOOKS_PATH=/content/drive/MyDrive/perceptionAI/TechGo/nvidia-tao
#FIXME4
%env EXPERIMENT_DIR=/content/drive/MyDrive/perceptionAI/TechGo/TAO/results/ssd
%env DATA_DIR=/content/drive/MyDrive/perceptionAI/TechGo/data/TechKITTI/
SPECS_DIR=f"{os.environ['COLAB_NOTEBOOKS_PATH']}/tensorflow/ssd/specs"
%env SPECS_DIR={SPECS_DIR}

%env EPOCH=010

In [ ]:
!mkdir -p $EXPERIMENT_DIR/experiment_dir_etlt/300

Importante considerations
- Set "data_type" to fp16 or fp32
- "gen_ds_config" will output a "nvinfer_config.txt" file that shows important configurations to take into account in the configuration file of the model in deepstream "tech_ssd_config.txt".

In [ ]:
!tao ssd export -m $EXPERIMENT_DIR/experiment_dir_retrain/300/weights/ssd_resnet10_epoch_010.tlt \
                         -o $EXPERIMENT_DIR/experiment_dir_etlt/300/ssd_resnet10_epoch_10_fp32.etlt \
                         -e $SPECS_DIR/300/ssd_retrain_resnet10_kitti.txt \
                         -k $KEY \
                         --data_type fp32 \
                         --gen_ds_config \
                         --target_opset 12

# MLFLOW

This section is for saving all the parameters and metrics of the trained model with mlflow. Remember to first import manually your "mlflow.db" database. If it is your first trained model, and you do not have a database, you can run in your local machine in cli "mlflow ui --backend-store-uri sqlite:///mlflow.db" and the database will be created.

In [ ]:
!pip install mlflow
!pip install boto3

We need to authenticate in order to save the artifacts into s3 bucket

In [ ]:
import os
!export AWS_SHARED_CREDENTIALS_FILE=/content/drive/MyDrive/config/awscli.ini
path = "/content/drive/MyDrive/config/awscli.ini"
os.environ['AWS_SHARED_CREDENTIALS_FILE'] = path
print(os.environ['AWS_SHARED_CREDENTIALS_FILE'])

In this case we will save the training configuration files as artifacts and some additional parameters.

In [6]:
# Params
pre_trained_weights = 'resnet10'
pruning_threshold = 0.1

In [7]:
import mlflow

expr_name = "object_detection"

s3_bucket = f"s3://elvis-s3-mlflow/mlruns/{expr_name}"

mlflow.set_tracking_uri("sqlite:///mlflow.db")
experiment = mlflow.get_experiment_by_name(expr_name)
if not experiment:
    mlflow.create_experiment(expr_name, s3_bucket)

mlflow.set_experiment(expr_name)

mlflow.start_run('64b6f7b983864931a88b766a90072562')
# mlflow.set_tag("mlflow.runName", 'ssdresnet10_fp16')

# Params
mlflow.log_param("pre_trained_weights", pre_trained_weights)
mlflow.log_param("pruning_threshold", pruning_threshold)

# Train params
mlflow.log_artifacts('/content/drive/MyDrive/perceptionAI/TechGo/nvidia-tao/tensorflow/ssd/specs/300', artifact_path="train_params")

# Evaluate
mlflow.end_run()